In [69]:
%pip install langchain langchain_community 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [75]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain.llms.openai import OpenAIChat
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.output_parsers.json import JsonOutputParser
import pandas as pd
from IPython.display import display, Markdown
llm = Ollama(model="llama3:latest", num_ctx=4096, num_predict=1024, temperature=0.1)
#llm = OpenAIChat(model="gpt-4", temperature=0.1)

In [76]:
question = "give me the supplychain powerdynamics map of the company ASML?"

# Tree of thoughts

In [77]:
import json

def load_graph(file):
    with open(file) as f:
        data = json.load(f)

    text = "```mermaid\ngraph LR\n"
    for item in data["links"]:
        text += f'\t{item["source"]} -- {item["value"]} --> {item["target"]}\n'
    text += "```"
    return text
text = load_graph("miserables_1.json")

graph_length = len(text)

print(f"Graph length: {graph_length}")
text = "no input data"

Graph length: 7352


```mermaid
graph LR;
    A[Process Inputs] --> B[Create Thoughts];
    B --> C[Evaluate Pathways];
    C --> D[Summarize Results];
    D --> E[Formulate Conclusions];
```

In [78]:
display(Markdown(text))
#display(text)

no input data

# Process inputs

Process the potential inputs into the the model: 

In [80]:
prompt_process_inputs = PromptTemplate.from_template("""
## 1. Process inputs:
    Given your knowledge, expressed as markdown format. 
    Please count the number of nodes and links in the graph.
### Question
{question}
                                                         
""")

chain  = prompt_process_inputs | llm | StrOutputParser()

Take the entire Graph 

In [81]:
processedData =  chain.invoke({
    "graph": text,
    "question": question
})

display(Markdown(processedData))

A fascinating question!

ASML (Applied Materials Sciences Ltd.) is a leading semiconductor equipment manufacturer. To create a supply chain relation map, I'll provide an overview of their key suppliers and customers.

**Nodes:**

1. **ASML**: The company itself, which is the central node in our graph.
2. **Suppliers:**
	* **Intel**: A major customer of ASML's semiconductor equipment.
	* **Samsung**: Another significant customer of ASML's products.
	* **TSMC** (Taiwan Semiconductor Manufacturing Company): A leading foundry that uses ASML's equipment.
	* **GlobalFoundries**: A leading independent semiconductor foundry that also uses ASML's equipment.
	* **Other suppliers**: Various companies providing components, materials, and services to ASML.

3. **Customers:**
	* **Intel**: Receives ASML's semiconductor equipment for their own manufacturing processes.
	* **Samsung**: Also receives ASML's equipment for their production lines.
	* **TSMC**: Uses ASML's equipment in their foundry operations.
	* **GlobalFoundries**: Similarly, uses ASML's equipment in their foundry operations.

**Links:**

1. **ASML → Suppliers**: ASML purchases components, materials, and services from these suppliers to manufacture its semiconductor equipment.
2. **Suppliers → ASML**: The suppliers provide the necessary goods and services to ASML for the production of their equipment.
3. **ASML → Customers**: ASML sells its semiconductor equipment to these customers, who use it in their manufacturing processes.

**Counting nodes and links:**

* Nodes: 7 (ASML, Intel, Samsung, TSMC, GlobalFoundries, Other suppliers)
* Links: 6 (ASML → Suppliers, Suppliers → ASML, ASML → Customers)

Please note that this is a simplified representation of the supply chain relation map for ASML. The actual graph would likely include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry.

Would you like me to elaborate on any specific aspect of this graph or provide additional information?

# List Toughts

In [83]:
prompt_create_toughts = ChatPromptTemplate.from_template("""
## 2. Create list of 10 toughts:
    Given the processed data, please create a list of toughts that you have about the data.
    Only list the that are relevant to the data and user question: {question}
    Do not answer the question yet.
    DO ONLY output as a valid 'JSON' array of strings in a JSON codeblock.
### Processed data
{data}
                                                         
### Question
{question}
                                                         
""")

chain_toughts  = prompt_create_toughts | llm | JsonOutputParser()

A string Array of JSON strings

In [84]:
listOfThoughts = chain_toughts.invoke({
    "data": processedData,
    "question": question
    })

In [85]:
display((listOfThoughts))

['The data appears to be a simplified representation of the supply chain relation map for ASML.',
 'There are 7 nodes in total: ASML, Intel, Samsung, TSMC, GlobalFoundries, and Other suppliers.',
 'The links represent the relationships between these nodes, with ASML being the central node.',
 'ASML purchases components, materials, and services from its suppliers to manufacture its semiconductor equipment.',
 'The suppliers provide goods and services to ASML for the production of their equipment.',
 'ASML sells its semiconductor equipment to Intel, Samsung, TSMC, and GlobalFoundries, who use it in their manufacturing processes.',
 'The actual graph would likely include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry.']

#### Loop over elements

## Evaluate Pathways

In [86]:
prompt_evaluate = ChatPromptTemplate.from_template("""
## 3. Evaluate pathways:
    Given the thoughts and the data, please evaluate and argue with evaluation criterias the pathways of each thought.
    DO NOT answer the question yet.
    DO ONLY format your output as 'markdown'.
                                                   
### Processed data
{data}
                                                   
### List of thoughts
{thoughts}
                                                         
### Question
{question}

""")

chain_evaluate  = prompt_evaluate | llm | StrOutputParser()
evaluation = chain_evaluate.invoke({
    "data": text,
    "thoughts": listOfThoughts,
    "question": question
    })

display(Markdown(evaluation))

### Evaluation Criteria for Pathways

* **Relevance**: How well does the pathway relate to the original question?
* **Accuracy**: Is the information presented accurate and consistent with available data?
* **Completeness**: Does the pathway provide a comprehensive overview of the supply chain relation map for ASML?

### Evaluation of Pathways

#### Thought 1: "The data appears to be a simplified representation of the supply chain relation map for ASML."

* Relevance: High
* Accuracy: Unclear (no input data provided)
* Completeness: Low

#### Thought 2: "There are 7 nodes in total: ASML, Intel, Samsung, TSMC, GlobalFoundries, and Other suppliers."

* Relevance: Medium
* Accuracy: Unclear (number of nodes not specified)
* Completeness: Medium

#### Thought 3: "The links represent the relationships between these nodes, with ASML being the central node."

* Relevance: High
* Accuracy: Unclear (no specific information about relationships)
* Completeness: Low

#### Thought 4: "ASML purchases components, materials, and services from its suppliers to manufacture its semiconductor equipment."

* Relevance: Medium
* Accuracy: Unclear (no specific data provided)
* Completeness: Medium

#### Thought 5: "The suppliers provide goods and services to ASML for the production of their equipment."

* Relevance: Medium
* Accuracy: Unclear (no specific data provided)
* Completeness: Medium

#### Thought 6: "ASML sells its semiconductor equipment to Intel, Samsung, TSMC, and GlobalFoundries, who use it in their manufacturing processes."

* Relevance: High
* Accuracy: Unclear (no specific data provided)
* Completeness: Low

#### Thought 7: "The actual graph would likely include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry."

* Relevance: Medium
* Accuracy: Unclear (no specific information about additional nodes and links)
* Completeness: High

## Synthesize Pathway Evaluation

In [87]:
prompt_synthesize_evaluate = ChatPromptTemplate.from_template("""
## 4. Evaluate pathways:
    Synthesize the data gathered from thoughts list and evaluation pathways
    Create a comprehensive synthesis for answering the question: {question}.
    DO NOT answer the question yet.
    DO process all thoughts
                                                   
### List of thoughts
{thoughts}

### Pathways evaluation
{evaluation}
                                                         
### Question
{question}

""")

chain_synthesize_evaluate  = prompt_synthesize_evaluate | llm | StrOutputParser()
synthesis = chain_synthesize_evaluate.invoke({
    "data": text,
    "thoughts": listOfThoughts,
    "evaluation": evaluation,
    "question": question
    })


In [88]:

display(Markdown(synthesis))

Based on the evaluation of pathways, it appears that Thought 7 has the highest relevance to answering the question "give me the supplychain relation map of the company ASML?" as it acknowledges the complexity and scope of the actual graph. However, since there is no specific information provided about additional nodes and links, the accuracy and completeness of this thought are unclear.

To answer the question, I would suggest combining Thoughts 2-6 to provide a comprehensive overview of the supply chain relation map for ASML. This could include:

* The 7 nodes mentioned in Thought 2: ASML, Intel, Samsung, TSMC, GlobalFoundries, and Other suppliers
* The relationships between these nodes, with ASML being the central node (Thought 3)
* The flow of goods and services from suppliers to ASML for production (Thoughts 4-5)
* The sales of semiconductor equipment by ASML to Intel, Samsung, TSMC, and GlobalFoundries (Thought 6)

However, it is important to note that this would likely be a simplified representation, as mentioned in Thought 7. A more comprehensive supply chain relation map for ASML would likely include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry.

Therefore, I would recommend providing a detailed and accurate representation of the supply chain relation map for ASML, including all relevant nodes and links, to provide a complete and comprehensive answer to the question.

## Formulate Conclusions

In [89]:
prompt_formulate = ChatPromptTemplate.from_template("""
## 5. Formulate answer:
    Given the synthesized data, please formulate a final answer to the question: {question}.
    DO be specific with names of companies, suppliers and products.
    DO give product names and ingredients.
    DO only formulate the final answer to the user.
    DO create a comprehensive and engaging article that supplychain professionals would find intresting.
    DO use 2000 words using markdown syntax, with sections and subsections.
    DO write a paragraph for each thought and evaluation pathline
    DO NOT give any URL's or links as references

### Thoughts                                   
{thoughts} 

### Evaluation                                   
{evaluation}

### Synthesis                                   
{synthesis}
                                                         
### Question
{question}

""")

chain_formulate  = prompt_formulate | llm | StrOutputParser()
formulatedoutput = chain_formulate.invoke({
    "synthesis": synthesis,
    "thoughts": listOfThoughts,
    "evaluation": evaluation,
    "question": question
    })

display(Markdown(formulatedoutput))

**Supply Chain Relation Map of ASML**

ASML Holding N.V. is a leading provider of semiconductor equipment for the global electronics industry. As one of the largest players in the sector, ASML's supply chain is complex and far-reaching, involving numerous companies and organizations around the world. In this article, we will provide an overview of the supply chain relation map for ASML, highlighting key nodes and relationships.

**Nodes**

1. **ASML**: The central node in our supply chain relation map, ASML is a leading provider of semiconductor equipment.
2. **Intel**: A major customer of ASML's products, Intel is one of the largest technology companies in the world.
3. **Samsung**: Another significant customer of ASML's products, Samsung is a South Korean conglomerate with a strong presence in the electronics industry.
4. **TSMC**: Taiwan Semiconductor Manufacturing Company (TSMC) is a leading independent semiconductor foundry and a major customer of ASML's products.
5. **GlobalFoundries**: A global leader in the development and manufacturing of advanced semiconductor technologies, GlobalFoundries is also a significant customer of ASML's products.
6. **Other Suppliers**: This node represents various other companies that supply goods and services to ASML for the production of its semiconductor equipment.

**Relationships**

ASML purchases components, materials, and services from its suppliers (Other Suppliers) to manufacture its semiconductor equipment. These suppliers provide a range of goods and services, including:

* **Components**: Such as silicon wafers, chemicals, and other raw materials used in the production process.
* **Materials**: Including packaging materials, substrates, and other supplies needed for manufacturing.
* **Services**: Like design, testing, and assembly services that support ASML's equipment development and manufacturing.

In turn, ASML sells its semiconductor equipment to Intel, Samsung, TSMC, and GlobalFoundries, who use it in their manufacturing processes. These customers rely on ASML's products to produce a wide range of electronic devices, including microprocessors, memory chips, and other components used in smartphones, laptops, and other consumer electronics.

**Additional Nodes and Links**

While the above nodes and relationships provide a comprehensive overview of ASML's supply chain relation map, it is important to note that this would likely be a simplified representation. In reality, the actual graph would include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry.

For example, additional suppliers might include:

* **Specialty Chemicals**: Companies that provide specialized chemicals used in ASML's manufacturing processes.
* **Packaging Materials**: Suppliers of packaging materials, such as substrates and encapsulants, used to protect and support ASML's products.
* **Assembly Services**: Providers of assembly services, like wire bonding and flip chip attachment, that support ASML's equipment development and manufacturing.

Similarly, additional customers might include:

* **Other Semiconductor Companies**: Firms that design, manufacture, and test their own semiconductor devices using ASML's equipment.
* **Original Equipment Manufacturers (OEMs)**: Companies that use ASML's products to produce electronic devices, such as smartphones, laptops, and servers.
* **Research Institutions**: Organizations that conduct research and development in the field of semiconductors, often relying on ASML's equipment for their work.

By acknowledging the complexity and scope of the actual graph, we can better appreciate the intricate relationships between companies, organizations, and industries involved in the supply chain relation map for ASML.

In [90]:
prompt_diagram = ChatPromptTemplate.from_template("""
## 5. Formulate answer:
    Given the synthesized data, please make a final answer to the question: {question}.
    DO be specific with names of companies, suppliers and products.
    DO give product names and ingredients.
    DO ONLY formulate the final answer to the user.
    DO ONLY produce a graviz diagram of the supplychain
    Make sure the graviz syntax is valid
   

### Thoughts                                   
{thoughts} 

### Evaluation                                   
{evaluation}

### Synthesis                                   
{synthesis}

### Formulated output                                   
{formulatedoutput}
                                                         
### Question
{question}

""")

chain_diagram  = prompt_diagram | llm | StrOutputParser()
diagram = chain_diagram.invoke({
    "synthesis": synthesis,
    "thoughts": listOfThoughts,
    "evaluation": evaluation,
    "formulatedoutput": formulatedoutput,
    "question": question
    })

display(Markdown(diagram))

Here is a Gviz diagram that represents the supply chain relation map of ASML:

```gviz
digraph ASML_Supply_Chain {
    node[shape=box, style=filled];
    "ASML" [label="ASML Holding N.V."];
    "Intel" [label="Intel Corporation"];
    "Samsung" [label="Samsung Electronics Co., Ltd."];
    "TSMC" [label="Taiwan Semiconductor Manufacturing Company (TSMC)"];
    "GlobalFoundries" [label="GlobalFoundries Inc."];
    "Other Suppliers" [label="Various Other Suppliers"];

    ASML -> Intel[label="Semiconductor Equipment Sales"];
    ASML -> Samsung[label="Semiconductor Equipment Sales"];
    ASML -> TSMC[label="Semiconductor Equipment Sales"];
    ASML -> GlobalFoundries[label="Semiconductor Equipment Sales"];
    Other Suppliers -> ASML[label="Components, Materials, and Services"];

    "Specialty Chemicals" [label="Specialty Chemicals Inc."];
    "Packaging Materials" [label="Packaging Materials Co., Ltd."];
    "Assembly Services" [label="Assembly Services Inc."];

    Other Suppliers -> Specialty Chemicals[label="Chemicals"];
    Other Suppliers -> Packaging Materials[label="Packaging Materials"];
    Other Suppliers -> Assembly Services[label="Assembly Services"];

    Intel -> ASML[label="Semiconductor Equipment Purchases"];
    Samsung -> ASML[label="Semiconductor Equipment Purchases"];
    TSMC -> ASML[label="Semiconductor Equipment Purchases"];
    GlobalFoundries -> ASML[label="Semiconductor Equipment Purchases"];
}
```

This Gviz diagram represents the supply chain relation map of ASML, showing the company's relationships with its suppliers and customers. The nodes in the graph represent companies, organizations, or industries involved in the supply chain, while the edges represent the flow of goods and services between these entities.

Please note that this is a simplified representation, as mentioned earlier, and the actual graph would likely include many more nodes and links, representing various other companies, organizations, and relationships involved in the semiconductor industry.

In [68]:
synthesis

"Based on the evaluation of the pathways, I will synthesize the data gathered from thoughts list and create a comprehensive synthesis for answering the question.\n\n**Supply Chain Relation Map of Unilever**\n\nThe central node in this supply chain is **Unilever**, which connects various nodes through different relationships. Here's a visual representation of the map:\n\n* **Manufacturing Facilities**: Connected to Unilever through production, indicating that these facilities produce goods for Unilever.\n* **Distributors**: Connected to Manufacturing Facilities for storage and transportation, suggesting that distributors receive products from manufacturing facilities and store them before transporting them to retailers.\n* **Retailers**: Connected to Distributors for product sales, implying that retailers purchase products from distributors and sell them to consumers.\n* **Farmers/Food Producers**: Connected to Unilever for raw material sourcing, indicating that Unilever sources raw mat